In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, classification_report
import os

: 

In [47]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size=5, hidden_layer_size=50, output_size=2):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.fc = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        out, (h_n, c_n) = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out
    
def create_sequences(data, seq_length, overlap):
    sequences = []
    result = []
    count = 0
    
    for i in range(0, len(data) - seq_length + 1, seq_length - overlap):
        count += 1
        sequence = data.iloc[i:i + seq_length]
        sequence = sequence.values
        sequences.append(sequence)

    return np.array(sequences)

In [48]:
data = pd.read_csv("./Capture/gokul.csv")
# data = data.drop(["dx", "dy"], axis=1)
data = data.fillna(-1)
data = data.replace(True, 1)
data = data.replace(False, 0)
data["timestamp"] = pd.to_datetime(data["timestamp"])
data["timestamp"] = data["timestamp"].diff().dt.total_seconds()
data["speed"] = np.sqrt(data["x_position"].diff() ** 2 + data["y_position"].diff() ** 2) / data["timestamp"]
data["moved"] = data["x_position"].diff() + data["y_position"].diff()
data = data.fillna(method="ffill").iloc[1:, :]
data = data.fillna(0)
data.reset_index(drop=True, inplace=True)

C:\Users\Gokul\AppData\Local\Temp\ipykernel_32640\1365990304.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace(True, 1)
C:\Users\Gokul\AppData\Local\Temp\ipykernel_32640\1365990304.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace(False, 0)
C:\Users\Gokul\AppData\Local\Temp\ipykernel_32640\1365990304.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill").iloc[1:, :

In [49]:
categories = [['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 
                  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 
                  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
                  '.', ',', '?', '!', ':', ';', '"', "'", '-', '(', ')', '[', ']', '{', '}', 
                  '+', '*', '/', '=', '<', '>', "ae", "#", "_", "|"
                  ]]

data["key"] = data["key"].replace(-1, "ae")
encoder = OneHotEncoder(categories=categories, sparse_output=False)
encoded = encoder.fit_transform(data[["key"]])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['key']))
df_encoded = pd.concat([data.drop('key', axis=1), encoded_df], axis=1)
df_encoded = df_encoded.drop("key_ae", axis=1)
df_encoded["button"] = df_encoded["button"].astype(float)
df_encoded["click"] = df_encoded["click"].astype(float)
df_encoded["press"] = df_encoded["press"].astype(float)

In [50]:
sequence = create_sequences(df_encoded, 50, 30)
sequence[np.isnan(sequence)] = 0

In [51]:
sequence = torch.from_numpy(sequence)
sequence = torch.tensor(sequence, dtype=torch.float32)

C:\Users\Gokul\AppData\Local\Temp\ipykernel_32640\2558704304.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequence = torch.tensor(sequence, dtype=torch.float32)


In [52]:
model = LSTMClassifier(input_size=94, hidden_layer_size=188, output_size=2)
model.load_state_dict(torch.load("model.pth"))
model.eval()

with torch.no_grad():
    prediction = model(sequence)

prediction_class = torch.argmax(prediction, dim=1)
print(prediction_class)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0])


C:\Users\Gokul\AppData\Local\Temp\ipykernel_32640\3172761160.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))
